Check out the test repo and figure out which time steps have data

In [13]:
import icechunk as ic
import xarray as xr

storage = ic.s3_storage(
    bucket="nasa-eodc-public",
    prefix="icechunk/MUR-JPL-L4-GLOB-v4.1-virtual-v2-p2",
    anonymous=True,
)
config = ic.Repository.fetch_config(storage=storage)
vchunk_containers = config.virtual_chunk_containers.keys()
container_credentials = ic.containers_credentials(
    {k: ic.s3_credentials(from_env=True) for k in vchunk_containers}
)
repo = ic.Repository.open(
    storage=storage, authorize_virtual_chunk_access=container_credentials
)
session = repo.readonly_session("main")
ds = xr.open_zarr(session.store, consolidated=False, zarr_format=3)

/Users/juliusbusecke/Code/titiler-multidim/.venv/lib/python3.12/site-packages/zarr/codecs/numcodecs/_codecs.py:139: ZarrUserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


In [14]:
ds

<xarray.Dataset> Size: 9TB
Dimensions:           (time: 497, lat: 17999, lon: 36000)
Coordinates:
  * time              (time) datetime64[ns] 4kB 2024-06-02T09:00:00 ... 2025-...
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
Data variables:
    analysed_sst      (time, lat, lon) float64 3TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 3TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 1TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 3TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
Attributes: (12/42)
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    acknowledgment:             Please acknowledge the use of these data with...
    easternmost_longitude:      180.0
    geospatial_lat_resolution:  0.009999999776482582
    netcdf_version_id:          4.1
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    history:                    created at nominal 4-day latency; replaced nr...
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    start_time:                 20240602T090000Z
    publisher_url:              http://www.ghrsst.org
    file_quality_level:         3

In [15]:
ds["analysed_sst"].isel(lon=slice(3000, 4000), lat=slice(5000, 6000), time=0).plot()

IcechunkError:   x error fetching virtual reference
  | 
  | context:
  |    0: icechunk::store::get
  |            with key="analysed_sst/c/0/5/1" byte_range=From(0)
  |              at icechunk/src/store.rs:198
  | 
  |-> error fetching virtual reference
  |-> service error
  |-> unhandled error (AccessDenied)
  `-> Error { code: "AccessDenied", message: "Access Denied", s3_extended_request_id: "C29JZSW7u+iryR07cSGIZmCB/c2xkXipcWHdKc/Z8L2HL3x6q1wC/FeDbKaYjk4y0K6CyrimfhJ5ldKQlDYWrg==", aws_request_id:
      "ZK5T2KFE8ZEGDZPG" }


## Test the tilejson endpoint 

Adopted from https://github.com/developmentseed/titiler-cmr/blob/develop/docs/examples/xarray_backend_example.ipynb

In [ ]:
import json

import httpx
import xarray as xr
from folium import Map, TileLayer

titiler_endpoint = (
    "https://syonjbeb09.execute-api.us-west-2.amazonaws.com"  # dev endpoint
)


variable = "Tair"
datetime_ = ds.time.values[0]
url = "s3://nasa-waterinsight/virtual-zarr-store/NLDAS-3-icechunk/"

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/WebMercatorQuad/tilejson.json",
    params=(
        ("url", url),
        # ("decode_times", True),
        # ("datetime", datetime_),
        ("variable", variable),
        ("rescale", "270,300"),
        ("colormap_name", "blues_r"),
        # ("sel_method", "nearest"),
    ),
).json()

print(r)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://syonjbeb09.execute-api.us-west-2.amazonaws.com/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?url=s3%3A%2F%2Fnasa-waterinsight%2Fvirtual-zarr-store%2FNLDAS-3-icechunk%2F&variable=Tair&rescale=270%2C300&colormap_name=blues_r'], 'minzoom': 1, 'maxzoom': 7, 'bounds': [-168.9999951170962, 7.000000114825381, -51.99999725350927, 71.99999511680303], 'center': [-110.49999618530273, 39.49999761581421, 1]}


In [11]:
bounds = r["bounds"]
m = Map(location=(70, -40), zoom_start=3)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="NASA",
).add_to(m)
m

In [6]:
geojson_dict = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "coordinates": [
                    [
                        [-20.79973248834736, 83.55979308678764],
                        [-20.79973248834736, 75.0115425216471],
                        [14.483337068956956, 75.0115425216471],
                        [14.483337068956956, 83.55979308678764],
                        [-20.79973248834736, 83.55979308678764],
                    ]
                ],
                "type": "Polygon",
            },
        }
    ],
}

r = httpx.post(
    f"{titiler_endpoint}/statistics",
    params=(
        ("url", url),
        ("datetime", datetime_),
        ("variable", variable),
    ),
    json=geojson_dict,
    timeout=60,
).json()

print(json.dumps(r, indent=2))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -20.79973248834736,
              83.55979308678764
            ],
            [
              -20.79973248834736,
              75.0115425216471
            ],
            [
              14.483337068956956,
              75.0115425216471
            ],
            [
              14.483337068956956,
              83.55979308678764
            ],
            [
              -20.79973248834736,
              83.55979308678764
            ]
          ]
        ]
      },
      "properties": {
        "statistics": {
          "2024-06-02T09:00:00.000000000": {
            "min": 271.34999999999997,
            "max": 279.005,
            "mean": 272.71998336321025,
            "count": 2841063.5,
            "sum": 774814790.4538239,
            "std": 2.1572893578676204,
            "median": 271.